In [1]:
# imports
from utils.pointcloud import BBox, grid_subsample_simple
from data.fwf_dataset import FwfDataset
from models.fgf import FGFeatNetwork
from omegaconf import OmegaConf
import tracemalloc

# tracemalloc.start()


import numpy as np
import json
import os


# build config
cfg = OmegaConf.load("./config/default.yaml")
cfg = OmegaConf.merge(cfg, OmegaConf.load(cfg.data.split))
with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
    cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))


print(OmegaConf.to_yaml(cfg))






general:
  device: cuda
  batch_size: 1024
  weight_decay: 0.0005
  max_epochs: 50
data:
  dataset_root: ../../02_Datasets/FWF_Subsampled/0.01/
  split: ./config/default_split.yaml
  preprocessing:
    _transformsTraining_:
      xyz:
        TransCenterXyz: {}
        TransZRotation: {}
        TransScaling: {}
        TransGaussianNoise:
          mean: 0
          std: 0.002
      rgb:
        TransGaussianNoise:
          mean: 0
          std: 0.02
        TransFeatureDropout:
          dropout_prob: 0.1
      wfm:
        TransSignalScaling: {}
      normals:
        TransGaussianNoise:
          mean: 0
          std: 0.05
      incAngles:
        TransGaussianNoise:
          mean: 0
          std: 0.05
        TransStandardize:
          mean: 2.07
          std: 0.38
      distanceFromScanner:
        TransGaussianNoise:
          mean: 12.13
          std: 12.2
    _transformsValidation_:
      xyz:
        TransCenterXyz: {}
      incAngles:
        TransStandardize:
      

In [2]:
train_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsTraining_, cfg.data._trainProjects_)
val_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsValidation_, cfg.data._valProjects_)


Loading '2024-03-22_FW_Koenigshuegel.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_02.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_03.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_04.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_05.FwfProj'; Bounding box IDs = default
Loading '2024-05-10_FW_RWTH_Zentrum_01.FwfProj'; Bounding box IDs = default
Loading '2024-07-31_FW_Bruecke_Koenigstr.FwfProj'; Bounding box IDs = [0, 2]
Loading '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Deltourserb.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj'; Bounding box IDs = [1]
Loading '2024-08-02_FW_Bruecke_RotheErde.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Rottstrasse.FwfProj'; Bounding box IDs = default
Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs 

In [3]:
# normals, inc angles
train_ds.subsample_grid(cfg.data.query_grid_size, save_inv=True)
train_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
train_ds.compute_normals_knn()
train_ds.compute_incAngles()


val_ds.subsample_grid(cfg.data.query_grid_size, save_inv=True)
val_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
val_ds.compute_normals_knn()
val_ds.compute_incAngles()


Computing neibors for '2024-03-22_FW_Koenigshuegel.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_03.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_04.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_05.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-05-10_FW_RWTH_Zentrum_01.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=000' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=002' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Deltourserb.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=001' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_RotheErde.FwfProj::defaultBbox'

In [4]:
from torch.utils.data import DataLoader

# create model
model = FGFeatNetwork(cfg=cfg,
    num_input_feats = train_ds[0]['features_neibors'].shape[-1],
    ).to(device=cfg.general.device)


In [5]:
# train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=0) --> ca. 6it/sec, no mem problem
# train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=2) --> ca. 12 it/sec, doable but still knapp

In [16]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch
from tqdm import tqdm
from utils.metrics import get_multilevel_metrics, print_metrics, combine_metrics_list
from time import time

criterion = CrossEntropyLoss()

optim = Adam(params=model.parameters(), weight_decay=cfg.general.weight_decay)

val_dl = DataLoader(val_ds, batch_size=cfg.general.batch_size, num_workers=4)

for epoch in range(cfg.general.max_epochs):
    print(f"Epoch-{epoch:03}")

    start = time()
    print(f'Sampling training dataset.',end=' ')
    train_ds.subsample_random(0.01)
    train_ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction, verbose=False)
    train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=4, pin_memory=True)
    print(f'Done. Took {time()-start:.2f}.')
    # train
    epoch_train_loss = []
    epoch_metrics = []
    model.train()
    for i, batch in enumerate(tqdm(train_dl, desc=f"{'Training':<15}", leave=True)):
        optim.zero_grad()
        # put batch on device
        for k, v in batch.items():
            batch[k] = v.to(device=cfg.general.device)

        # forward pass
        out = model(batch)
        
        # agregate loss on all levels
        loss = torch.tensor(0.).to(device=cfg.general.device)
        for k in out.keys():
            output = out[k]
            gt = batch[k] # type:ignore
            loss += criterion(output, gt)
        
        loss.backward()
        optim.step()


        # aggregate values for metric calculation
        epoch_train_loss.append(loss.item())


        preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}    
        epoch_metrics.append(get_multilevel_metrics(preds, batch, cfg))
        del batch, out
        torch.cuda.empty_cache()
        



    epoch_metrics = combine_metrics_list(epoch_metrics,cfg)
    print_metrics(epoch_metrics, cfg)
    print(f"L_train:{np.mean(epoch_train_loss):.4f} ")
    
    # validate  
    epoch_metrics = []
    epoch_val_loss = []
    model.eval()
    with torch.no_grad():
        for i,batch in enumerate(tqdm(val_dl, desc=f"{'Validation':<15}", leave=True)):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=cfg.general.device)

            # forward pass
            out = model(batch)
            
            # agregate loss on all levels
            loss = torch.tensor(0.).to(device=cfg.general.device)
            for k in out.keys():
                output = out[k]
                gt = batch[k] # type:ignore
                loss += criterion(output, gt)

        # aggregate values for metric calculation
        epoch_val_loss.append(loss.item())
        preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}    
        epoch_metrics.append(get_multilevel_metrics(preds, batch, cfg))
        del batch, out
        torch.cuda.empty_cache()
            
    epoch_metrics = combine_metrics_list(epoch_metrics,cfg)
    print_metrics(epoch_metrics, cfg)


    print(f"L_val:{np.mean(epoch_val_loss):.4f}")
    print('\n')

    

Epoch-000
Sampling training dataset. Done. Took 22.67.


Training       : 100%|██████████| 885/885 [07:43<00:00,  1.91it/s]


                oA      mIoU        mP        mR      mF1     mcAcc
labels_0  0.906056  0.497311  0.547246  0.549102  0.53911  0.549102
L_train:0.2393 


Validation     : 100%|██████████| 843/843 [00:22<00:00, 37.36it/s]


                oA      mIoU        mP        mR       mF1     mcAcc
labels_0  0.782875  0.260958  0.333333  0.260958  0.292739  0.260958
L_val:0.6410


Epoch-001
Sampling training dataset. Done. Took 23.47.


...

                oA      mIoU        mP        mR       mF1     mcAcc
labels_0  0.930683  0.310228  0.333333  0.310228  0.321366  0.310228
L_val:0.2191


Epoch-049
Sampling training dataset. Done. Took 22.75.


Training       : 100%|██████████| 885/885 [07:28<00:00,  1.98it/s]


                oA      mIoU        mP        mR       mF1     mcAcc
labels_0  0.965222  0.685658  0.766195  0.723284  0.732392  0.723284
L_train:0.1072 


Validation     : 100%|██████████| 843/843 [00:21<00:00, 39.72it/s]

                oA      mIoU        mP        mR       mF1     mcAcc
labels_0  0.924567  0.308189  0.333333  0.308189  0.320268  0.308189
L_val:0.2315




In [ ]:
# Epoch-049
# Sampling training dataset. Done. Took 22.75.
# Training       : 100%|██████████| 885/885 [07:28<00:00,  1.98it/s]
#                 oA      mIoU        mP        mR       mF1     mcAcc
# labels_0  0.965222  0.685658  0.766195  0.723284  0.732392  0.723284
# L_train:0.1072 
# Validation     : 100%|██████████| 843/843 [00:21<00:00, 39.72it/s]
#                 oA      mIoU        mP        mR       mF1     mcAcc
# labels_0  0.924567  0.308189  0.333333  0.308189  0.320268  0.308189
# L_val:0.2315